### Treinamento e Validação com a Base Inicial:

In [2]:
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import numpy as np
import evaluate

In [3]:
# --- 1. CARREGAR E PREPARAR O SEED SET ANOTADO ---
caminho_do_arquivo = '/Users/giossaurus/Developer/leia_tcc/data/processed/seed_set_anotado.csv'
df_anotado = pd.read_csv(caminho_do_arquivo)
df_anotado = df_anotado.rename(columns={'question': 'text', 'intent_choice': 'label'})

In [4]:
# Mapear os rótulos de texto para números (IDs)
labels = df_anotado['label'].unique().tolist()
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}
df_anotado['label'] = df_anotado['label'].map(label2id)

# Criar o dataset e configurar a coluna label como ClassLabel
from datasets import ClassLabel
full_seed_dataset = Dataset.from_pandas(df_anotado)

# Converter a coluna label para ClassLabel para permitir estratificação
features = full_seed_dataset.features.copy()
features['label'] = ClassLabel(names=labels)
full_seed_dataset = full_seed_dataset.cast(features)

Casting the dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

In [5]:
# --- 2. DIVISÃO ESTRATÉGICA DOS DADOS ---
# Dividir os 200 exemplos em Treino (80%) e Teste Final (20%)
dataset_split = full_seed_dataset.train_test_split(test_size=0.2, stratify_by_column="label", seed=42)

# Este é o nosso núcleo de treino, que vai crescer a cada ciclo
seed_train_dataset = dataset_split["train"]

# ESTE É O NOSSO CONJUNTO DE TESTE FINAL. NÃO USAREMOS AGORA.
final_test_dataset = dataset_split["test"]
# Salvar para uso futuro no final do projeto
final_test_dataset.to_csv('/Users/giossaurus/Developer/leia_tcc/data/processed/final_test_set.csv', index=False)


print("--- DADOS ESTRATEGICAMENTE DIVIDIDOS ---")
print(f"Núcleo de Treinamento (Seed Train): {len(seed_train_dataset)} exemplos")
print(f"Conjunto de Teste Final (Guardado): {len(final_test_dataset)} exemplos")

# Criar um conjunto de validação a partir do núcleo de treino para monitorar este ciclo
train_val_split = seed_train_dataset.train_test_split(test_size=0.2, stratify_by_column="label", seed=42)
train_dataset = train_val_split["train"]
eval_dataset = train_val_split["test"]

print(f"-> Para este Ciclo 1: Treino = {len(train_dataset)}, Validação = {len(eval_dataset)}")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

--- DADOS ESTRATEGICAMENTE DIVIDIDOS ---
Núcleo de Treinamento (Seed Train): 160 exemplos
Conjunto de Teste Final (Guardado): 40 exemplos
-> Para este Ciclo 1: Treino = 128, Validação = 32


In [6]:
# --- 3. PREPARAR O MODELO E TOKENIZADOR ---
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(
    model_checkpoint, 
    num_labels=len(labels), 
    id2label=id2label, 
    label2id=label2id
)

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/32 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# --- 4. CONFIGURAR E EXECUTAR O TREINAMENTO ---
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    acc = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")
    
    return {"accuracy": acc["accuracy"], "f1": f1["f1"]}

training_args = TrainingArguments(
    output_dir="./models/leia_classifier_cycle1",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

print("\n--- INICIANDO TREINAMENTO DO MODELO v1 ---")
trainer.train()
print("\n--- TREINAMENTO CONCLUÍDO ---")

/var/folders/k4/x1x0j4qs0svb48fpvhhvkmbh0000gn/T/ipykernel_13211/771060541.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



--- INICIANDO TREINAMENTO DO MODELO v1 ---


/opt/homebrew/Caskroom/miniforge/base/envs/torch-mps/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.276618,0.375000,0.204545
2,No log,1.218083,0.375000,0.204545
3,No log,1.143623,0.593750,0.459150
4,No log,1.092624,0.593750,0.459150
5,No log,1.074675,0.625000,0.483254


/opt/homebrew/Caskroom/miniforge/base/envs/torch-mps/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/homebrew/Caskroom/miniforge/base/envs/torch-mps/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/homebrew/Caskroom/miniforge/base/envs/torch-mps/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/homebrew/Caskroom/miniforge/base/envs/torch-mps/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then de


--- TREINAMENTO CONCLUÍDO ---


In [8]:
# Salvar o modelo e o tokenizador deste ciclo
trainer.save_model("./models/leia_classifier_v1_final")
print("Modelo v1 salvo em './models/leia_classifier_v1_final'")

Modelo v1 salvo em './models/leia_classifier_v1_final'
